In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch import optim, Tensor
import math
from tqdm.notebook import tqdm
from chords_dataset import ChordsDataset
from model_helpers import NLP
from model import Transformer

device = torch.device("cpu" if torch.cuda.is_available() else "cpu")

df = pd.read_csv('./model_data/lyrics_processed.csv')
dataset = ChordsDataset(df, NLP)
train_i, test_i, validation_i = dataset.get_train_test_valid_indexes(0.9,0.09,0.01)


LYR_VOCAB_SIZE = 100
EMB_SIZE = LYR_VOCAB_SIZE
CHORD_SIZE = len(dataset.chords_set) 
num_heads = 5
num_encoder_layers = 3
num_decoder_layers = 3
forward_expansion = 1
dropout = 0.1 
max_length = 7038

model = Transformer(EMB_SIZE,LYR_VOCAB_SIZE,CHORD_SIZE,15,num_heads,num_encoder_layers, num_decoder_layers, forward_expansion, dropout, max_length,device ).to(device)
optimizer = optim.Adam(model.parameters(), lr = 3e-4)
criterion = nn.CrossEntropyLoss().to(device)


c:\Users\bench\anaconda3\envs\p38\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\bench\anaconda3\envs\p38\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\bench\anaconda3\envs\p38\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %
c:\Users\bench\anaconda3\envs\p38\lib\site-packages\torchtext\data\utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [2]:
# def output_to_chords(result_tensor: Tensor):

def eval_model(dataset: ChordsDataset,validation_indexes:list, model: nn.Module, criterion: nn.Module):
    model.eval()
    loss = 0
    nb_elem = len(validation_indexes)
    for index in validation_indexes:
        elem = dataset[index]
        source, target = elem["lyrics"].reshape(-1,1,100).to(device), elem["chords"].reshape(-1,1)
        output = model(source, target).reshape(-1,CHORD_SIZE)
        target_one_hot= Tensor([dataset.to_one_hot(int(chord)) for chord in target]).to(device)
        loss += criterion(output, target_one_hot)
    model.train()
    return loss/nb_elem


c:\Users\bench\Documents\EPFL\Master\MA4\Computador e musica\project\ComputadorMusica\chords_dataset.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
  "lyrics": Tensor([self.vectorize(word) for word in  self.lyrics[idx] ]),


tensor(7.9959, grad_fn=<DivBackward0>)

In [3]:


NB_EPOCHS = 3

for epoch in range(NB_EPOCHS):
    print(f"[Epoch {epoch}/ {NB_EPOCHS}]")
    model.train()
    for i,index in tqdm(enumerate(train_i)):
        elem = dataset[index]
        source, target = elem["lyrics"].reshape(-1,1,100).to(device), elem["chords"].reshape(-1,1)
        output = model(source, target).reshape(-1,CHORD_SIZE)
        optimizer.zero_grad()
        target_one_hot=  torch.Tensor([dataset.to_one_hot(int(chord)) for chord in target]).to(device)
        loss = criterion(output, target_one_hot)
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), max_norm= 1)
        optimizer.step()
        if i % 500 == 0:
            print(f"At i = {i} : ",eval_model(dataset, validation_i, model, criterion))

[Epoch 0/ 3]


C:\Users\bench\AppData\Local\Temp/ipykernel_32484/2463333720.py:14: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), max_norm= 1)


KeyboardInterrupt: 

In [49]:
import numpy as np 
from math import floor, ceil
l = np.array(range(15457))
np.random.shuffle(l)
l
# n = len(l)
# prop1 = 0.79
# prop2 = 0.11
# prop3 = 0.1
# indexes_1 = l[0:floor(n*prop1)]
# indexes_2 = l[floor(n*prop1):(floor(n*prop1) + floor(n*prop2))]
# indexes_3 = l[(floor(n*prop1) + floor(n*prop2)):n]
# print(len(indexes_1) + len(indexes_2)+  len(indexes_3))

array([  918,  2688, 14279, ...,  8961, 15203, 12957])